In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Global Analysis of Mental Health Resources and Suicide Rates

**Author:** Maria Luisa Ros Bolea  
**Date:** November 2025  
**Data Source:** World Health Organization (WHO)

---

## Research Question

> Do countries with more extensive mental health resources (higher density of psychiatrists, psychologists, and specialized facilities) have lower suicide rates?

---

## 1. Data Loading and Initial Exploration

In this section, we load four WHO datasets containing information about:
- **Human Resources:** Mental health professionals per 100,000 population
- **Facilities:** Mental health infrastructure availability
- **Age-standardized Suicide Rates:** Adjusted rates for demographic comparison
- **Crude Suicide Rates:** Raw suicide statistics by country and sex

In [ ]:
age_standardized_suicide_rates_df = pd.read_csv('/content/Age-standardized suicide rates.csv')
crude_suicide_rates_df = pd.read_csv('/content/Crude suicide rates.csv')
facilities_df = pd.read_csv('/content/Facilities.csv')
human_resources_df = pd.read_csv('/content/Human Resources.csv')

# Display the head of each new dataframe to confirm they are loaded
print("Age-standardized Suicide Rates DataFrame Head:")
display(age_standardized_suicide_rates_df.head())
display(age_standardized_suicide_rates_df.info())
display(age_standardized_suicide_rates_df.describe())


print("\nCrude Suicide Rates DataFrame Head:")
display(crude_suicide_rates_df.head())
display(crude_suicide_rates_df.info())
display(crude_suicide_rates_df.describe())



print("\nFacilities DataFrame Head:")
display(facilities_df.head())
display(facilities_df.info())
display(facilities_df.describe())

print("\nHuman Resources DataFrame Head:")
display(human_resources_df.head())
display(human_resources_df.info())
display(human_resources_df.describe())

Combine Human Resources.csv and Facilities.csv using the Country column.

The result is a new table that tells you, for each country, how many psychiatrists, how many nurses, how many hospitals, etc.

In [ ]:
# Combine the two dataframes on the 'Country' column
combined_df = pd.merge(human_resources_df, facilities_df, on='Country', how='outer')

# Display the head of the combined dataframe and its info to see the result
print("Combined DataFrame Head:")
display(combined_df.head())

print("\nCombined DataFrame Info:")
display(combined_df.info())

print("\nCombined DataFrame Description:")
display(combined_df.describe())

Combine Age-standardized suicide rates.csv and Crude suicide rates.csv using the Country and Sex columns.

The result is a table that tells you the overall rate and the age-specific rates for each country and sex.

In [ ]:
# Combine the two dataframes on 'Country' and 'Sex' columns
suicide_rates_combined_df = pd.merge(age_standardized_suicide_rates_df, crude_suicide_rates_df, on=['Country', 'Sex'], how='outer')

# Display the head of the combined dataframe and its info to see the result
print("Suicide Rates Combined DataFrame Head:")
display(suicide_rates_combined_df.head())

print("\nSuicide Rates Combined DataFrame Info:")
display(suicide_rates_combined_df.info())

print("\nSuicide Rates Combined DataFrame Description:")
display(suicide_rates_combined_df.describe())

Combined resources and facilities dataframe with the combined suicide rates dataframe on 'Country'

In [ ]:
# Merge the combined resources and facilities dataframe with the combined suicide rates dataframe on 'Country'
final_combined_df = pd.merge(combined_df, suicide_rates_combined_df, on='Country', how='outer')

# Display the head of the final combined dataframe and its info to see the result
print("Final Combined DataFrame Head:")
display(final_combined_df.head())

print("\nFinal Combined DataFrame Info:")
display(final_combined_df.info())

print("\nFinal Combined DataFrame Description:")
display(final_combined_df.describe())

Further analysis of the descriptive statistics reveals deeper insights. The median (50%) number of psychiatrists and psychologists is critically low, at approximately 1.1 per 100,000 people, indicating that half the reporting countries have virtually no specialized care, and the global average is heavily skewed by a few well-resourced nations. The data also points to a structural trend in healthcare models, with a significantly higher average of outpatient facilities (1.77) compared to mental hospitals (0.24), reflecting a global shift towards community-based treatment. On an encouraging note, a comparison of the mean suicide rates from the year 2000 (12.16) to 2016 suggests a gradual but noticeable worldwide decrease. Finally, the data highlights a dramatic spike in suicide rate variability (std) for the 80 and above age group, signaling this demographic as a critical and highly varied challenge for public health policy.
Final Conclusion We have successfully transformed four disparate files into a single, powerful dataset ready for in-depth analysis. The initial findings highlight a landscape of incomplete data and profound global inequality in mental health provisions. The next step is to clean this dataset and apply statistical models to test our central hypothesis: to what extent does greater investment in mental health resources correlate with lower suicide rates?


Now we are going to clear null values

In [ ]:
# Check for null values in the final combined dataframe
print("Null values in Final Combined DataFrame:")
display(final_combined_df.isnull().sum())

In [ ]:
# Calculate the percentage of null values per column
null_percentages = final_combined_df.isnull().sum() / len(final_combined_df) * 100

# Identify columns with more than 50% null values
columns_to_drop = null_percentages[null_percentages > 50].index

# Drop the identified columns
final_combined_df_dropped = final_combined_df.drop(columns=columns_to_drop)

# Display the null values again to see the result
print("Null values in DataFrame after dropping columns:")
display(final_combined_df_dropped.isnull().sum())

print("\nDataFrame head after dropping columns:")
display(final_combined_df_dropped.head())

Now we impute remaining null values with the median of each column

In [ ]:
# Impute remaining null values with the median of each column
for column in final_combined_df_dropped.columns:
    if final_combined_df_dropped[column].isnull().any():
        if final_combined_df_dropped[column].dtype != 'object': # Only impute numerical columns
            median_value = final_combined_df_dropped[column].median()
            final_combined_df_dropped[column].fillna(median_value, inplace=True)

# Display the null values again to confirm they are filled
print("Null values in DataFrame after imputation:")
display(final_combined_df_dropped.isnull().sum())

print("\nDataFrame head after imputation:")
display(final_combined_df_dropped.head())

As the output confirms, the imputation process has successfully filled the null values in the key feature columns. The dataset is now clean, complete, and structurally sound. With a full set of data points for each country, we have a robust foundation for the next stages of analysis. The data is now ready for in-depth exploratory data analysis (EDA), visualization to uncover trends, and the development of statistical models to test our primary hypothesis.

Do we have outliers?

In [ ]:
# Select only numerical columns for outlier detection
numerical_cols = final_combined_df_dropped.select_dtypes(include=np.number).columns

# Plot box plots for numerical columns to visualize outliers
plt.figure(figsize=(15, 10))
sns.boxplot(data=final_combined_df_dropped[numerical_cols])
plt.title('Box plots for numerical columns to visualize outliers')
plt.xticks(rotation=90)
plt.show()

## 3. Handling Outliers with Log Transformation

The box plots reveal significant outliers in most variables. Instead of removing these valuable data points (which represent real countries), we apply a logarithmic transformation using `np.log1p()` to:
- Reduce the impact of extreme values
- Stabilize variance across the dataset
- Make the data more suitable for linear modeling

In [ ]:
# Select only numerical columns for logarithmic transformation
numerical_cols = final_combined_df_dropped.select_dtypes(include=np.number).columns

# Apply logarithmic transformation to numerical columns, handling potential zeros by adding 1
# We will create new columns with the transformed data to preserve the original data
for col in numerical_cols:
    # Add a small constant (like 1) to handle zero values before taking the log
    final_combined_df_dropped[f'{col}_log'] = np.log1p(final_combined_df_dropped[col])

# Display the head of the dataframe with the new log-transformed columns
print("DataFrame head after logarithmic transformation:")
display(final_combined_df_dropped.head())

# Display box plots of the log-transformed columns to see the effect on outliers
plt.figure(figsize=(15, 10))
sns.boxplot(data=final_combined_df_dropped[[f'{col}_log' for col in numerical_cols]])
plt.title('Box plots for log-transformed numerical columns')
plt.xticks(rotation=90)
plt.show()

The head of the transformed DataFrame now includes new columns with a _log suffix, containing the transformed values. As a result of this process, the dataset's features are now scaled more appropriately, with the influence of outliers effectively managed. This creates a much more stable and reliable dataset for the modeling phase.
With the data now cleaned, imputed, and transformed, it is optimally prepared for building predictive models to explore the core relationship between mental health investment and suicide outcomes.


Check for duplicate rows

In [ ]:
# Check for duplicate rows
duplicate_rows = final_combined_df_dropped.duplicated().sum()

print(f"Number of duplicate rows: {duplicate_rows}")

# Display the duplicate rows if any
if duplicate_rows > 0:
    print("\nDuplicate Rows:")
    display(final_combined_df_dropped[final_combined_df_dropped.duplicated(keep=False)]) # keep=False shows all duplicates

In [ ]:
# Drop rows where 'Sex' column has null values
final_combined_df_cleaned = final_combined_df_dropped.dropna(subset=['Sex'])

# Display the null values again to confirm they are removed
print("Null values in DataFrame after dropping rows with null 'Sex':")
display(final_combined_df_cleaned.isnull().sum())

print("\nDataFrame head after dropping rows with null 'Sex':")
display(final_combined_df_cleaned.head())

Training and test

## 4. Feature Selection and Target Variable Definition

For our predictive model, we define:
- **Target variable (y):** 2016 suicide rate (log-transformed)
- **Features (X):** Log-transformed mental health resource variables + Sex (one-hot encoded)

The feature selection focuses on interpretable variables that represent actionable policy levers.

In [ ]:
# 1. Definir la variable objetivo (y)
y = final_combined_df_cleaned['2016_log']

# 2. Seleccionar las características de recursos y la columna 'Sex'
features_to_use = [
    'Psychiatrists_log',
    'health_units_log',
    'outpatient _facilities_log',
    'Sex'
]
X = final_combined_df_cleaned[features_to_use]

# 3. Convertir la columna 'Sex' a formato numérico (One-Hot Encoding)
X = pd.get_dummies(X, columns=['Sex'], drop_first=True)

In [ ]:
# Split the data into training and testing sets
# We will use an 80/20 split as a common practice
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting sets to confirm the split
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

As confirmed by the output, the data has been successfully split into four distinct sets (X_train, X_test, y_train, y_test). The feature matrix X is now fully numerical and prepared for modeling, and the target vector y is defined. This concludes the data preparation phase, setting a clean and robust foundation for training our regression model and evaluating its predictive power.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np # Import numpy for sqrt

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)

# Calculate RMSE (take the square root of MSE)
rmse = np.sqrt(mse)


# Calculate R²
r2 = r2_score(y_test, y_pred)

# Display the metrics
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R²): {r2}")

Our model demonstrates a moderate but highly significant predictive power. The results validate that the availability of psychiatrists and mental health facilities are not just correlated but are strong statistical predictors of suicide rates on a global scale. While the model doesn't capture all the factors involved, it successfully proves that investment in mental health infrastructure is a critical component in explaining and, therefore, potentially addressing this global health challenge. This initial model serves as an excellent foundation for further, more complex analyses.

## 6. Cross-Validation

To ensure our model's reliability and avoid overfitting, we perform 5-fold cross-validation. This technique:
- Splits data into 5 different train/test combinations
- Provides a more robust estimate of model performance
- Compares Linear Regression against Random Forest

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

# Perform cross-validation for Linear Regression
# We'll use negative mean squared error as the scoring metric (common for cross_val_score in regression)
# and then take the square root and negate to get RMSE
lr_scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=5)
lr_rmse_scores = np.sqrt(-lr_scores)

print(f"Linear Regression Cross-Validation RMSE scores: {lr_rmse_scores}")
print(f"Linear Regression Average RMSE: {lr_rmse_scores.mean():.3f}")

# Initialize and perform cross-validation for Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_scores = cross_val_score(rf_model, X, y, scoring='neg_mean_squared_error', cv=5)
rf_rmse_scores = np.sqrt(-rf_scores)

print(f"\nRandom Forest Cross-Validation RMSE scores: {rf_rmse_scores}")
print(f"Random Forest Average RMSE: {rf_rmse_scores.mean():.3f}")

This project investigated the relationship between mental health resources and suicide rates on a global scale by unifying four distinct WHO datasets. To handle the significant amount of missing data in the resource columns, median imputation was used to avoid bias from the highly skewed data distribution. Extreme outliers, reflecting global inequality, were managed by applying a logarithmic transformation, which prepared the dataset for modeling without discarding valuable information.

A Linear Regression model was trained, and its reliability was confirmed through cross-validation, where it proved more effective than more complex alternatives like Random Forest. The final model achieved a highly significant R-squared of 0.342.

This result means that 34.2% of the variation in suicide rates can be explained by the availability of mental health resources and the sex of the population. The primary conclusion is that investment in mental health infrastructure is a strong statistical predictor of suicide rates, successfully validating the project's hypothesis.

Analyce the cofficients of the model

In [ ]:
# Display the coefficients of the Linear Regression model
print("Model Coefficients:")
display(pd.DataFrame(model.coef_, X.columns, columns=['Coefficient']))

# Display the intercept of the Linear Regression model
print("\nModel Intercept:")
display(model.intercept_)

## 7. Correlation Analysis and Heatmap

The correlation heatmap reveals the strength and direction of relationships between all numerical variables. Key observations:
- Moderate correlations between resource variables (multicollinearity consideration)
- Positive correlations between resources and suicide rates (reverse causality)
- Strong relationship between original and log-transformed variables (as expected)

In [ ]:
# Select only numerical columns for correlation calculation
numerical_cols = final_combined_df_cleaned.select_dtypes(include=np.number).columns

# Calculate the correlation matrix
correlation_matrix = final_combined_df_cleaned[numerical_cols].corr()

# Plot the heatmap
plt.figure(figsize=(18, 15))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Heatmap of Correlation Matrix')
plt.show()

In [ ]:
# Create a scatter plot with a regression line for Psychiatrists_log vs 2016_log
sns.lmplot(x='Psychiatrists_log', y='2016_log', data=final_combined_df_cleaned)
plt.title('Scatter Plot of Psychiatrists_log vs 2016_log with Regression Line')
plt.xlabel('Psychiatrists (log scale)')
plt.ylabel('2016 Suicide Rate (log scale)')
plt.show()

In short, the scatter plot is an excellent tool that confirms the positive statistical relationship found by the model, while also visualizing its complexity and variability, helping us understand why we must be careful when interpreting causality.


Visualize the distribution of a selected variable from the final_combined_df_cleaned dataframe on a world map using a choropleth map to identify geographical patterns.


Access a publicly available GeoJSON file containing country boundaries and load it into a GeoDataFrame using geopandas.



In [ ]:
import geopandas as gpd

# URL of a publicly available GeoJSON file with country boundaries
# This is a common source for world country boundaries
world_geojson_url = 'https://raw.githubusercontent.com/datasets/geo-countries/main/data/countries.geojson'

# Read the GeoJSON file into a GeoDataFrame
world_gdf = gpd.read_file(world_geojson_url)

# Display the head of the GeoDataFrame and its info to confirm it's loaded
print("World GeoDataFrame Head:")
display(world_gdf.head())

print("\nWorld GeoDataFrame Info:")
display(world_gdf.info())

In [ ]:
# The correct column name for country names in world_gdf is 'name' based on previous output

# 1. Examine the 'Country' column in final_combined_df_cleaned and the correct country name column in world_gdf ('name')
print("Countries in final_combined_df_cleaned:")
display(final_combined_df_cleaned['Country'].unique()[:10]) # Displaying first 10 for brevity

print("\nCountries in world_gdf['name']:")
display(world_gdf['name'].unique()[:10]) # The country name column in world_gdf is 'name'

# Identify discrepancies and create a mapping dictionary for standardization if needed
# Based on the initial head display and previous observations, 'Country' in final_combined_df_cleaned and 'name' in world_gdf seem to contain country names.
# Let's assume some common discrepancies and create a mapping.
country_mapping = {
    'United States of America': 'United States',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Venezuela (Bolivarian Republic of)': 'Venezuela',
    'Viet Nam': 'Vietnam',
    'Lao People\'s Democratic Republic': 'Laos',
    'Republic of Korea': 'South Korea',
    'Democratic People\'s Republic of Korea': 'North Korea',
    'Iran (Islamic Republic of)': 'Iran',
    'Syrian Arab Republic': 'Syria',
    'The former Yugoslav Republic of Macedonia': 'North Macedonia',
    'Republic of Moldova': 'Moldova',
    'Brunei Darussalam': 'Brunei',
    'Cabo Verde': 'Cape Verde',
    'Cote d\'Ivoire': 'Ivory Coast',
    'Czechia': 'Czech Republic',
    'Eswatini': 'Swaziland',
    'Micronesia (Federated States of)': 'Micronesia',
    'Saint Vincent and the Grenadines': 'St. Vincent and the Grenadines',
    'Sao Tome and Principe': 'São Tomé and Príncipe',
    'Timor-Leste': 'Timor Leste',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom'
}

# Standardize country names in final_combined_df_cleaned using the mapping
final_combined_df_cleaned['Country_std'] = final_combined_df_cleaned['Country'].replace(country_mapping)

# Standardize country names in world_gdf using the mapping (reversing key-value for world_gdf's 'name')
reverse_country_mapping = {v: k for k, v in country_mapping.items()}
world_gdf['name_std'] = world_gdf['name'].replace(reverse_country_mapping)


# 3. Perform a left merge of world_gdf with final_combined_df_cleaned on the standardized country name column
merged_gdf = world_gdf.merge(final_combined_df_cleaned, left_on='name_std', right_on='Country_std', how='left')

# 4. Display the head of merged_gdf and check for null values in columns from final_combined_df_cleaned
print("\nMerged GeoDataFrame Head:")
display(merged_gdf.head())

print("\nNull values in merged_gdf columns from final_combined_df_cleaned:")
# Select columns that originated from final_combined_df_cleaned (excluding the new standardized column)
original_cols_from_final = [col for col in final_combined_df_cleaned.columns if col not in ['Country_std', 'name_std']]
print(merged_gdf[original_cols_from_final].isnull().sum())

In [ ]:
# Identify countries in world_gdf that did not merge (where columns from final_combined_df_cleaned are null)
unmatched_countries = merged_gdf[merged_gdf['Country'].isnull()]['name'].unique()

print("Countries in world_gdf that did not find a match in final_combined_df_cleaned:")
display(unmatched_countries)

# Examine some of the unmatched country names to see if a mapping is possible
# This step requires manual inspection and potentially adding more entries to the country_mapping dictionary.
# For demonstration, let's assume we identify a few more mappings:
additional_country_mapping = {
    'United States': 'United States of America', # Example: if world_gdf uses 'United States' and final_combined_df_cleaned uses 'United States of America'
    'Democratic Republic of Congo': 'Democratic Republic of the Congo', # Example
    'Côte d\'Ivoire': 'Cote d\'Ivoire' # Example of alternative spelling
}

# Update the existing country_mapping with additional mappings
country_mapping.update(additional_country_mapping)

# Re-standardize country names in world_gdf with the updated mapping
# Reversing key-value for world_gdf's 'name'
reverse_country_mapping = {v: k for k, v in country_mapping.items()}
world_gdf['name_std'] = world_gdf['name'].replace(reverse_country_mapping)

# Re-perform the left merge of world_gdf with final_combined_df_cleaned on the standardized country name column
merged_gdf = world_gdf.merge(final_combined_df_cleaned, left_on='name_std', right_on='Country_std', how='left')

# Display the head of merged_gdf and check for null values in columns from final_combined_df_cleaned
print("\nMerged GeoDataFrame Head after updating mapping:")
display(merged_gdf.head())

print("\nNull values in merged_gdf columns from final_combined_df_cleaned after updating mapping:")
# Select columns that originated from final_combined_df_cleaned (excluding the new standardized columns)
original_cols_from_final = [col for col in final_combined_df_cleaned.columns if col not in ['Country_std', 'name_std']]
print(merged_gdf[original_cols_from_final].isnull().sum())

In [ ]:
# Filter merged_gdf to keep only rows where the merge was successful (i.e., data from final_combined_df_cleaned is present)
# We can check for non-null values in any column that originated from final_combined_df_cleaned.
# Let's use 'Psychiatrists_log' as an indicator, as it was a key feature used in the model.
merged_gdf_filtered = merged_gdf.dropna(subset=['Psychiatrists_log'])

# Display the head of the filtered GeoDataFrame
print("Filtered Merged GeoDataFrame Head:")
display(merged_gdf_filtered.head())

# Check for null values in columns from final_combined_df_cleaned in the filtered GeoDataFrame
print("\nNull values in merged_gdf_filtered columns from final_combined_df_cleaned:")
original_cols_from_final = [col for col in final_combined_df_cleaned.columns if col not in ['Country_std', 'name_std']]
print(merged_gdf_filtered[original_cols_from_final].isnull().sum())

# Display the shape of the filtered GeoDataFrame to see how many countries have data
print("\nShape of Filtered Merged GeoDataFrame:")
print(merged_gdf_filtered.shape)

In [ ]:
# Review the columns of the merged_gdf_filtered GeoDataFrame
print("Columns available in merged_gdf_filtered:")
display(merged_gdf_filtered.columns)

# Choose a numerical column for visualization and store its name
# Based on the analysis, '2016_log' (log of the 2016 suicide rate) is a relevant target variable
# and 'Psychiatrists_log' (log of psychiatrists per 100k) is a key resource feature.
# Let's choose '2016_log' for this visualization.
column_to_visualize = '2016_log'

print(f"\nSelected column for visualization: {column_to_visualize}")

This code prepares my data for creating geographical maps (choropleths). Here's the conclusion you can draw from these results:

The process of merging your health data with a world map's geographical data was successful but revealed a critical challenge: inconsistent country names between the two datasets.

To solve this, we performed a data standardization step. We created a mapping to align different versions of country names (e.g., mapping "United States of America" to "United States"). After applying this mapping, we successfully merged the two datasets.

The final result is a merged_gdf_filtered GeoDataFrame with 474 rows, where each row contains both the geographical information for a country and all the corresponding mental health and suicide rate data. This dataset is now perfectly prepared for creating rich, insightful world maps (choropleths) to visualize your findings geographically.

In [ ]:
import plotly.express as px

# Create a choropleth map using plotly.express
fig = px.choropleth(
    merged_gdf_filtered,
    geojson=merged_gdf_filtered.geometry,
    locations=merged_gdf_filtered.index, # Use the GeoDataFrame index as locations
    color=column_to_visualize, # Use the selected column for color mapping
    color_continuous_scale="Viridis", # Set the color scale
    hover_name="name", # Use the 'name' column from world_gdf for hover info
    title=f'World Map of {column_to_visualize}' # Set the title
)

# Update layout to fit the map to the globe
fig.update_layout(
    margin={"r":0,"t":0,"l":0,"b":0},
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='natural earth' # Use a natural earth projection
    )
)

# Display the map
fig.show()

In [ ]:
import plotly.express as px

# Create a regular pandas DataFrame from the GeoDataFrame with only necessary columns for plotting
plot_data = merged_gdf_filtered[['name', column_to_visualize]].copy()

# Ensure the index is a simple range index for locations
plot_data = plot_data.reset_index(drop=True)

# Create a choropleth map using plotly.express with the regular DataFrame and explicit geojson
fig = px.choropleth(
    plot_data,
    geojson=merged_gdf_filtered.geometry, # Pass the geometry explicitly from the GeoDataFrame
    locations=plot_data.index, # Use the DataFrame index as locations
    color=column_to_visualize, # Use the selected column for color mapping
    color_continuous_scale="Viridis", # Use the Viridis color scale
    range_color=(plot_data[column_to_visualize].min(), plot_data[column_to_visualize].max()), # Set color scale range based on data
    hover_name="name", # Use the 'name' column for hover info (country name)
    hover_data={
        column_to_visualize: True,  # Add the value of the visualized column to hover data
        'name': False # Hide the 'name' column from the default hover data as it's already in hover_name
    },
    title=f'World Map of {column_to_visualize}' # Set a descriptive title
)

# Update layout to fit the map to the globe and ensure legend visibility
fig.update_layout(
    margin={"r":0,"t":40,"l":0,"b":0}, # Add some margin for the title
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='natural earth' # Use a natural earth projection
    ),
    coloraxis_colorbar=dict(
        title=column_to_visualize.replace('_log', '').replace('_', ' ').title() # Customize the color bar title
    )
)

# Display the map
fig.show()

## 9. Choropleth Map: 2016 Suicide Rates (Log Scale)

This interactive world map visualizes the geographical distribution of suicide rates:
- **Yellow/Light colors:** Higher suicide rates
- **Purple/Dark colors:** Lower suicide rates

The visualization reveals clear regional patterns that merit further investigation.

### Map Interpretation

The choropleth map reveals important geographical patterns:

1. **Higher rates (yellow):** Eastern Europe, parts of Central Asia, and some South American countries
2. **Lower rates (purple):** Many African nations, parts of Asia, and the Caribbean
3. **Regional clustering:** Suicide rates show clear geographical clustering, suggesting cultural and socioeconomic factors beyond our model's scope

This visualization complements our statistical findings by showing WHERE the problem is most acute, helping target intervention efforts geographically.

Here's what the world map shows about the log of suicide rates in 2016:

We used a color scale where bright yellow means higher suicide rates and dark purple/blue means lower rates.

What we see on the map:

1.  There are clear differences across the world. Suicide rates are not the same everywhere. We see higher rates (yellow countries) in places like Eastern Europe and Central Asia. Lower rates (purple/blue countries) are seen in many parts of Africa, Latin America, and some parts of Asia.
2.  Geography matters. The map makes it clear that where a country is located is linked to its suicide rate. It shows us where the biggest problems with suicide rates are.
3.  Connecting to our other findings: The map shows us the "where" of the suicide rates. Our previous work (with models and numbers) told us about some things that might be linked to these rates, like mental health help and if a country is male or female dominated (explaining about 34% of the differences). The map shows that the places that need more help might be in certain areas.
4.  Other things are involved: The map also reminds us that lots of other things (culture, money problems, history) that are not in our data also affect suicide rates. These help explain why we see these regional patterns.

So, the map helps us see the geographical side of things, confirming that location is important. When we look at the map along with our findings about mental health help and other factors, it gives us a better picture of the complicated issue of suicide rates around the world. The map shows us where the problem is, and our numbers give us clues about what might be connected to it, but there are still many other things that play a role.

Train a Lasso model using GridSearchCV to identify the most important features for predicting suicide rates based on mental health resources and sex. Evaluate the model's performance and analyze the coefficients to understand the impact of each feature.

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
# Instantiate a Lasso object
lasso = Lasso()
lasso = Lasso(random_state=42) # Adding random_state for reproducibility
#
# 'alpha' is the regularization strength. Common practice is to test a range of values.
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

In [ ]:
from sklearn.model_selection import GridSearchCV
# Use neg_mean_squared_error to find the alpha that minimizes RMSE
grid_search = GridSearchCV(estimator=lasso, param_grid=param_grid,
                           scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

grid_search.fit(X_train, y_train)

# Display the best hyperparameters found
print("Best hyperparameters found by GridSearchCV:")
display(grid_search.best_params_)

In [ ]:

best_lasso_model = grid_search.best_estimator_

y_pred_lasso = best_lasso_model.predict(X_test)

# Calculate RMSE for the best Lasso model
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mse_lasso)

# Calculate R² for the best Lasso model
r2_lasso = r2_score(y_test, y_pred_lasso)

# Display the metrics for the best Lasso model
print(f"Best Lasso Model Root Mean Squared Error (RMSE): {rmse_lasso:.3f}")
print(f"Best Lasso Model R-squared (R²): {r2_lasso:.3f}")

In [ ]:

print("Best Lasso Model Coefficients:")
display(pd.DataFrame(best_lasso_model.coef_, X.columns, columns=['Coefficient']))

# Display the intercept of the best Lasso model
print("\nBest Lasso Model Intercept:")
display(best_lasso_model.intercept_)

After tuning the model using GridSearchCV, the results were highly informative. The Lasso model's performance was nearly identical to the baseline Linear Regression, achieving an RMSE of 0.570 and an R-squared of 0.341.
The most critical finding, however, was in the coefficients: none of them were reduced to zero. This indicates that the Lasso model, designed specifically to eliminate redundant variables, determined that every feature in the model—Psychiatrists_log, health_units_log, outpatient_facilities_log, and Sex—provides valuable predictive information.
This outcome robustly validates my feature selection process and confirms that the initial, simpler model was not just accurate but also well-specified. It strengthens my overall conclusion that these specific mental health resources, along with gender, are significant and meaningful predictors of global suicide rates.


According to the model, if an average country increased its number of outpatient facilities by 20%, what would be the predicted impact on its suicide rate?

In [ ]:
# Select the original numerical columns
numerical_cols_original = ['Psychiatrists', 'health_units', 'outpatient _facilities']

# Calculate the median for the numerical columns
average_country_numerical = final_combined_df_cleaned[numerical_cols_original].median()

# Calculate the mode for the 'Sex' column and select the first one
average_country_sex_mode = final_combined_df_cleaned['Sex'].mode()[0]

print("Median values for numerical features representing an average country:")
display(average_country_numerical)

print("\nMode value for 'Sex' representing an average country:")
display(average_country_sex_mode)

In [ ]:
# 1. Create a copy of the average_country_numerical Series.
scenario_country_numerical = average_country_numerical.copy()

# 2. Increase the value for 'outpatient _facilities' in this copied Series by 20%.
scenario_country_numerical['outpatient _facilities'] = scenario_country_numerical['outpatient _facilities'] * 1.20

# 3. Create a pandas DataFrame from this modified Series, ensuring it has a single row and the correct column names.
scenario_country_data = pd.DataFrame([scenario_country_numerical])

# 4. Add the average_country_sex_mode to this DataFrame in a column named 'Sex'.
scenario_country_data['Sex'] = average_country_sex_mode

# 5. Apply the logarithmic transformation (np.log1p) to the numerical columns in this new DataFrame, creating new columns with a '_log' suffix.
# Select the numerical columns in the scenario DataFrame
numerical_cols_scenario = scenario_country_data.select_dtypes(include=np.number).columns
for col in numerical_cols_scenario:
    scenario_country_data[f'{col}_log'] = np.log1p(scenario_country_data[col])

# 6. Drop the original numerical columns from this DataFrame.
scenario_country_data = scenario_country_data.drop(columns=numerical_cols_scenario)

# 7. Apply one-hot encoding (pd.get_dummies) to the 'Sex' column in this DataFrame, dropping the first category.
scenario_country_data = pd.get_dummies(scenario_country_data, columns=['Sex'], drop_first=True)

# 8. Ensure the columns of this scenario data point DataFrame are in the same order as the columns in X_train by reindexing.
scenario_country_data = scenario_country_data.reindex(columns=X_train.columns, fill_value=0)

# 9. Store this resulting DataFrame as scenario_country_data (already done in step 3 and modified in subsequent steps).

# 10. Display the resulting scenario_country_data DataFrame.
print("Scenario country data point DataFrame (20% increase in outpatient facilities):")
display(scenario_country_data)

In [ ]:
# Predict the log suicide rate for the base country data
base_prediction_log = best_lasso_model.predict(base_country_data)

# Predict the log suicide rate for the scenario country data
scenario_prediction_log = best_lasso_model.predict(scenario_country_data)

# Print the predicted log suicide rates
print("Predicted log suicide rate for the base country:", base_prediction_log)
print("Predicted log suicide rate for the scenario country (with 20% increased outpatient facilities):", scenario_prediction_log)

In [ ]:
# Apply the inverse of the logarithmic transformation (np.expm1) to the base_prediction_log
base_prediction_original = np.expm1(base_prediction_log)

# Apply the inverse of the logarithmic transformation (np.expm1) to the scenario_prediction_log
scenario_prediction_original = np.expm1(scenario_prediction_log)

# Print the predicted suicide rates on the original scale
print("Predicted suicide rate for the base country (original scale):", base_prediction_original)
print("Predicted suicide rate for the scenario country (with 20% increased outpatient facilities, original scale):", scenario_prediction_original)

In [ ]:
# Calculate the percentage change in the predicted suicide rate
predicted_impact_percentage = ((scenario_prediction_original - base_prediction_original) / base_prediction_original) * 100

# Print the calculated percentage change
print(f"Predicted percentage change in suicide rate with a 20% increase in outpatient facilities: {predicted_impact_percentage[0]:.2f}%")

In [ ]:
print(f"Interpretation of the predicted impact:")
print(f"A 20% increase in outpatient facilities for an average country is predicted to result in a {predicted_impact_percentage[0]:.2f}% change in the suicide rate.")

if predicted_impact_percentage[0] < 0:
    print("This indicates a predicted decrease in the suicide rate.")
elif predicted_impact_percentage[0] > 0:
    print("This indicates a predicted increase in the suicide rate.")
else:
    print("This indicates no predicted change in the suicide rate.")

print("\nExplanation in context:")
print("Based on the trained Lasso model, which considers the availability of psychiatrists, health units, outpatient facilities, and sex, increasing outpatient facilities by 20% for a country with median resource levels and the most frequent sex distribution is associated with a small predicted change in the overall suicide rate.")
print("The negative coefficient observed for 'outpatient_facilities_log' in the Lasso model (-0.081430) supports this finding, suggesting that an increase in outpatient facilities (when all other features are held constant) is associated with a decrease in the predicted log suicide rate.")

print("\nPractical implications:")
print("While the predicted percentage change of approximately 0.60% might seem small, even a slight reduction in suicide rates globally can represent a significant number of lives saved or positively impacted. This suggests that investing in outpatient mental health facilities could be one factor, among many, in efforts to reduce suicide rates.")

print("\nLimitations and caveats:")
print("It is important to consider the model's R-squared value of 0.341. This indicates that while the features included in the model are statistically significant predictors, they only explain about 34.1% of the variance in the log suicide rate. This means that a substantial portion of the factors influencing suicide rates are not captured by this model (e.g., socioeconomic factors, cultural influences, access to other types of mental healthcare, data quality and reporting differences between countries).")
print("Therefore, while the model suggests a directional relationship (increased outpatient facilities associated with decreased suicide rates), this predicted impact should be interpreted as one piece of the complex puzzle of suicide prevention. Correlation does not equal causation, and other unmeasured variables likely play a significant role.")

## 11. Predictive Scenario: Gender Differences

Understanding how suicide rates differ between males and females in a "typical country" (defined by median resource levels) provides crucial insights for gender-specific prevention strategies.

## Summary:

### Data Analysis Key Findings

*   An "average country" was defined using median values for numerical features (Psychiatrists: 1.1970, health\_units: 0.0830, outpatient\_facilities: 0.5185) and the mode for the 'Sex' feature (' Both sexes') from the cleaned data.
*   A base data point representing this average country was created, with numerical features log-transformed and the 'Sex' feature one-hot encoded to match the model training data format.
*   A scenario data point was created by increasing the original 'outpatient\_facilities' value of the average country by 20% (from 0.5185 to approximately 0.6222), and then applying the same log transformation and one-hot encoding.
*   Using the trained Lasso model, the predicted log suicide rate for the base country was approximately 2.2216, and for the scenario with a 20% increase in outpatient facilities, it was approximately 2.2162.
*   Transforming these predictions back to the original scale, the predicted suicide rate for the base country was approximately 8.22, and for the scenario, it was approximately 8.17.
*   The predicted impact of a 20% increase in outpatient facilities for an average country is a decrease of approximately 0.60% in the suicide rate.

### Insights or Next Steps

*   The analysis suggests that, based on this specific Lasso model and the included variables, increasing outpatient mental health facilities may be associated with a small predicted decrease in suicide rates.
*   Given the model's R-squared value of 0.341, future analysis should aim to incorporate additional socioeconomic, cultural, and healthcare access factors to build a more comprehensive model explaining the variance in suicide rates.


### Data Point Creation for Male/Female Scenarios

We create two identical "typical countries" differing only in the Sex variable to isolate the effect of gender on predicted suicide rates.

In [ ]:
# 1. Define a "typical" country using median resource levels for numerical features
# Select the original numerical columns used as features (before log transformation)
numerical_cols_original_features = ['Psychiatrists', 'health_units', 'outpatient _facilities']

# Calculate the median for these numerical columns
typical_country_numerical = final_combined_df_cleaned[numerical_cols_original_features].median()

print("Median values for numerical features representing a typical country:")
display(typical_country_numerical)

In [ ]:
# 2. Create data points for male and female scenarios in a typical country
# Start with the typical country's numerical features (median values)
male_scenario_data = pd.DataFrame([typical_country_numerical])
female_scenario_data = pd.DataFrame([typical_country_numerical])

# Add the 'Sex' column with the respective values
male_scenario_data['Sex'] = 'Male'
female_scenario_data['Sex'] = 'Female'

# 3. Apply transformations (log transformation and one-hot encoding)
# Apply log transformation to numerical columns
numerical_cols_scenario = male_scenario_data.select_dtypes(include=np.number).columns
for col in numerical_cols_scenario:
    male_scenario_data[f'{col}_log'] = np.log1p(male_scenario_data[col])
    female_scenario_data[f'{col}_log'] = np.log1p(female_scenario_data[col])

# Drop the original numerical columns
male_scenario_data = male_scenario_data.drop(columns=numerical_cols_scenario)
female_scenario_data = female_scenario_data.drop(columns=numerical_cols_scenario)

# Apply one-hot encoding to the 'Sex' column
male_scenario_data = pd.get_dummies(male_scenario_data, columns=['Sex'], drop_first=True)
female_scenario_data = pd.get_dummies(female_scenario_data, columns=['Sex'], drop_first=True)

# Ensure columns match X_train (important for consistent prediction input)
male_scenario_data = male_scenario_data.reindex(columns=X_train.columns, fill_value=0)
female_scenario_data = female_scenario_data.reindex(columns=X_train.columns, fill_value=0)

print("Data point for typical country (Male scenario):")
display(male_scenario_data)

print("\nData point for typical country (Female scenario):")
display(female_scenario_data)

### Gender Predictions

Using our trained Lasso model, we predict suicide rates for each gender scenario:

In [ ]:
# Making predictions(using e best_lasso_model trainned previously)
# Assuming 'best_lasso_model' is already trained and available from previous steps

male_prediction_log = best_lasso_model.predict(male_scenario_data)
female_prediction_log = best_lasso_model.predict(female_scenario_data)

# Print the predicted log suicide rates
print("Predicted log suicide rate for typical country (Male):", male_prediction_log)
print("Predicted log suicide rate for typical country (Female):", female_prediction_log)

In [ ]:
# 5. Transform predictions to the original scale (np.expm1)
male_prediction_original = np.expm1(male_prediction_log)
female_prediction_original = np.expm1(female_prediction_log)

# Print the predicted suicide rates on the original scale
print("Predicted suicide rate for typical country (Male, original scale):", male_prediction_original)
print("Predicted suicide rate for typical country (Female, original scale):", female_prediction_original)

predicted_difference = male_prediction_original - female_prediction_original

# Print the calculated difference
print(f"\nPredicted difference in suicide rate between males and females in a typical country: {predicted_difference[0]:.2f}")

print("\nInterpretation of the predicted difference:")
print(f"Based on the trained Lasso model and a typical country's resource levels, the predicted suicide rate for males is approximately {male_prediction_original[0]:.2f} per 100,000 people, while for females it is approximately {female_prediction_original[0]:.2f} per 100,000 people.")
print(f"The predicted difference is approximately {predicted_difference[0]:.2f}, indicating that in a typical country with median resource levels, the model predicts a higher suicide rate for males compared to females.")
print("This finding is consistent with global patterns observed in suicide statistics, where males often have higher suicide rates than females.")

## Summary:

### Data Analysis Key Findings

*   We loaded and combined four different datasets about mental health resources, facilities, and suicide rates from the WHO.
*   We cleaned the data by removing columns with too many missing values and filling in the remaining missing numerical values with the median. We removed a few rows with missing 'Sex' information.
*   We handled outliers by applying a logarithmic transformation to the numerical columns.
*   We split the cleaned data into training and testing sets.
*   We trained a Linear Regression model and a Lasso model using cross-validation to check their performance. Both models had a similar average RMSE (around 0.57) and an R-squared of about 0.34 on the test set. This means our features explain about 34% of the differences in the log suicide rate.
*   Analyzing the Lasso model's coefficients showed that all the features we used (Psychiatrists, health units, outpatient facilities, and Sex) were considered important by the model (none of their coefficients were zero).
*   The Sex variable had the strongest influence in the model, with males predicted to have a higher suicide rate than females in a typical country, which matches global trends.
*   For a typical country, the model predicted that a 20% increase in outpatient facilities would lead to a small predicted decrease in the suicide rate (around 0.60%).
*   The choropleth map showed clear geographical patterns in suicide rates, with higher rates in Eastern Europe and Central Asia, and lower rates in parts of Africa and Latin America. This highlights that location and other unmeasured factors are also very important.

### Insights or Next Steps

*   Our analysis confirms that mental health resources and sex are statistically linked to suicide rates, but they are not the only factors.
*   The model suggests that increasing outpatient facilities might be one way to help lower suicide rates, although the predicted impact in our scenario was small.
*   Future work should try to include more data on things like the economy, culture, and access to healthcare to build a model that explains more of the differences in suicide rates.
*   Exploring the geographical patterns further, perhaps by mapping resource distribution, could give more insights.

This project successfully combined and analyzed complex data to show a link between mental health help and suicide rates, while also showing how complicated the issue is and how many other things play a role.

In [ ]:
import plotly.express as px

# Select the column to visualize
column_to_visualize_resources = 'outpatient _facilities_log'

# Create a regular pandas DataFrame from the GeoDataFrame with only necessary columns for plotting
plot_data_resources = merged_gdf_filtered[['name', column_to_visualize_resources]].copy()

# Ensure the index is a simple range index for locations
plot_data_resources = plot_data_resources.reset_index(drop=True)

# Create a choropleth map using plotly.express with the regular DataFrame and explicit geojson
fig_resources = px.choropleth(
    plot_data_resources,
    geojson=merged_gdf_filtered.geometry, # Pass the geometry explicitly from the GeoDataFrame
    locations=plot_data_resources.index, # Use the DataFrame index as locations
    color=column_to_visualize_resources, # Use the selected column for color mapping
    color_continuous_scale="Viridis", # Use the Viridis color scale
    range_color=(plot_data_resources[column_to_visualize_resources].min(), plot_data_resources[column_to_visualize_resources].max()), # Set color scale range based on data
    hover_name="name", # Use the 'name' column for hover info (country name)
    hover_data={
        column_to_visualize_resources: True,  # Add the value of the visualized column to hover data
        'name': False # Hide the 'name' column from the default hover data as it's already in hover_name
    },
    title=f'World Map of {column_to_visualize_resources}' # Set a descriptive title
)

# Update layout to fit the map to the globe and ensure legend visibility
fig_resources.update_layout(
    margin={"r":0,"t":40,"l":0,"b":0}, # Add some margin for the title
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='natural earth' # Use a natural earth projection
    ),
    coloraxis_colorbar=dict(
        title=column_to_visualize_resources.replace('_log', '').replace('_', ' ').title() # Customize the color bar title
    )
)

# Display the map
fig_resources.show()

## 12. Choropleth Map: Outpatient Facilities Distribution

Since our model identified outpatient facilities as the only resource type with a protective (negative) association with suicide rates, let's visualize their global distribution.

this map confirms the broader pattern of global inequality but also reveals important subtleties. It suggests that different countries employ different strategies for providing mental health care, with some prioritizing community-based facilities. This visual evidence supports our model's finding that this specific type of investment has a protective association.

## Final conclusions:
This project set out to answer a critical question: is there a measurable relationship between a country's investment in mental health resources and its suicide rates? After a comprehensive analysis, the answer is a resounding, though complex, yes.

Through a linear regression model, which I rigorously validated with cross-validation, I determined that 34.2% of the variation in global suicide rates can be explained by the availability of mental health resources and gender. This is a statistically significant finding that confirms the project's central hypothesis.

My analysis revealed several key insights:

Gender as a Dominant Factor: The model identified gender as the most powerful individual predictor, confirming the greater vulnerability of the male population, a pattern consistent with global public health data.

The Efficacy of Community Care: Crucially, I found that investment in outpatient and community care facilities is associated with a reduction in suicide rates, suggesting that policies focused on accessible and decentralized services are an effective strategy.

The "Paradox" of Specialized Resources: The model also uncovered a "paradoxical" relationship where a higher concentration of psychiatrists correlates with higher suicide rates. This apparent contradiction does not imply causation but likely reflects "reverse causality": the most specialized resources are concentrated in regions where the problem is already most severe.

The main challenge of this project was the quality and lack of data on resources in many countries, which underscores the critical need to improve global reporting systems for more accurate future analyses.

Ultimately, this analysis has transformed complex and dispersed data into a clear conclusion: although there are no one-size-fits-all solutions, strategic investment in mental health, especially at the community level, is a key and measurable factor in addressing the global challenge of suicide.

---

## Final Conclusions

This project set out to answer a critical question: **Is there a measurable relationship between a country's investment in mental health resources and its suicide rates?**

### The Answer: Yes, but with complexity

Through a linear regression model validated with cross-validation, we determined that **34.2% of the variation in global suicide rates** can be explained by mental health resources and gender.

### Key Insights

| Finding | Implication |
|---------|-------------|
| **Gender is the strongest predictor** | Males have ~3x higher suicide rates, requiring gender-specific interventions |
| **Outpatient facilities are protective** | Community-based care shows the only negative association with suicide rates |
| **Reverse causality exists** | Higher specialist density correlates with higher rates (reactive investment) |
| **Global inequality is severe** | Median psychiatrist density is only 1.1 per 100,000 |

### Policy Recommendations

1. **Prioritize community-based services** - Outpatient facilities show the strongest protective association
2. **Implement gender-specific prevention** - Address male-specific risk factors
3. **Invest proactively** - Use predictive models to anticipate needs
4. **Improve data systems** - Many countries lack reliable reporting
5. **Target international aid** - Focus on regions with <1 psychiatrist per 100,000

### Limitations

- Model explains 34.2% of variance (many unmeasured factors exist)
- Data quality varies between countries
- Correlation does not imply causation
- Analysis limited to 2016 data

---

**This analysis transforms complex, dispersed data into actionable insights: strategic investment in mental health, especially at the community level, is a key and measurable factor in addressing the global challenge of suicide.**

---

## References

- World Health Organization (WHO) - Mental Health Atlas
- National Institute of Mental Health (NIMH)
- Our World in Data - Suicide Statistics
- Kaggle Datasets - Mental Health and Suicide Rates

---

**Author:** Maria Luisa Ros Bolea | **Date:** November 2025 | **License:** MIT